# 📊 **Performance of ReAct Agent Running a Sample Question from TriviaQA Benchmark**


Import essential modules

In [ ]:
import os
import openai
import dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain_core.language_models.chat_models import BaseChatModel
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.react import (
    TRIVIAQA_FEWSHOT_EXAMPLES,
    REACT_INSTRUCTION_TRIVIAQA,
)
print("Environment has been set up")

In [ ]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("API key not found. Please check your .env file.")

print(f"API key has been activated: {api_key}")

In [ ]:
gpt_turbo = ChatOpenAI(api_key=api_key,model="gpt-3.5-turbo")
print("agent initialized")
agent = ReActAgent(llm=gpt_turbo)

In [ ]:
question = "American Callan Pinckney's eponymously named system became a best-selling (1980s-2000s) book/video franchise in what genre?"
outputs = agent.generate(question=question, examples=TRIVIAQA_FEWSHOT_EXAMPLES,
        prompt=REACT_INSTRUCTION_TRIVIAQA)

print(f"Question: {question}") 

if outputs:
    for output in outputs:
        print("Thought:", output.thought)
        print("Action:", output.action)
        print("Observation:", output.observation)
else:
    print("No output generated.")